In [ ]:
! pip install -q langchain
! pip install -q openai
! pip install python-dotenv
! pip install -q pypdf
! pip install -U langchain-community
! pip install -q chromadb
! pip install tiktoken
! pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49

In [ ]:
from langchain.document_loaders import PyPDFLoader


In [ ]:
from google.colab import files
import pandas as pd

print("Téléchargez vos fichiers PDF et CSV")
files_uploaded = files.upload()



Téléchargez vos fichiers PDF et CSV


Saving af-plc-investor-presentation-2023.pdf to af-plc-investor-presentation-2023.pdf
Saving africa-ir-pack-q4-2023.pdf to africa-ir-pack-q4-2023.pdf
Saving consensus-fy-2023.pdf to consensus-fy-2023.pdf
Saving FY 2023 Communiqué de Presse.pdf to FY 2023 Communiqué de Presse.pdf
Saving FY 2023 Comptes consolidés - Excel.xlsx to FY 2023 Comptes consolidés - Excel.xlsx
Saving FY 2023 Consolidated Financial Statement - Excel.xlsx to FY 2023 Consolidated Financial Statement - Excel.xlsx
Saving FY 2023 Liste des filiales et participations.pdf to FY 2023 Liste des filiales et participations.pdf
Saving FY 2023 Présentation -FR.pdf to FY 2023 Présentation -FR.pdf
Saving MTN-Group-FY-23-Annual-Financial-Statement.pdf to MTN-Group-FY-23-Annual-Financial-Statement.pdf
Saving MTN-Group-FY-23-results-SENS-1.pdf to MTN-Group-FY-23-results-SENS-1.pdf
Saving Orange-COMFI-Groupe-Sonatel-FY2023-BRVM-FR.pdf to Orange-COMFI-Groupe-Sonatel-FY2023-BRVM-FR.pdf
Saving Orange-CP_Groupe-OCI-Resultats-2023.pdf t

In [ ]:
data = []
for filename in files_uploaded.keys():
    if filename.endswith('.pdf'):
        loader = PyPDFLoader(filename)
        data.extend(loader.load())

csv_data = []
for filename in files_uploaded.keys():
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        csv_data.append(df)

print(f"{len(data)} documents PDF chargés")

771 documents PDF chargés


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ''

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
question = "What is the work dress code for male employees?"
docs = vectorstore.similarity_search(question)

print(f"Question length: {len(docs)}, document length: {len(data)}, words in first document: {len(data[0].page_content.split())}")

Question length: 4, document length: 771, words in first document: 13


In [ ]:
import logging
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(),
    llm=ChatOpenAI(temperature=0)
)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)
print(len(unique_docs))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the dress code policy for men in the workplace?', '2. What are the guidelines for professional attire for male staff members?', "3. How should male employees dress according to the company's standards?"]


8


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())

In [ ]:
question = "what is all documents about?"
qa_chain({"query": question})

{'query': 'what is all documents about?',
 'result': "I don't know, could you please provide more context or specify which documents you are referring to?"}

In [ ]:
question = "quelle est le Total du passif et des capitaux propres de la france en 2022?"
qa_chain({"query": question})

{'query': 'quelle est le Total du passif et des capitaux propres de la france en 2022?',
 'result': 'Le total du passif et des capitaux propres de la France en 2022 était de 2 252 192 210 milliers de FCFA.'}

In [ ]:
question = "quel est les flux nets de trésorerie générés par l'activité (a) en 2023 ?"
qa_chain({"query": question})

{'query': "quel est les flux nets de trésorerie générés par l'activité (a) en 2023 ?",
 'result': "Les flux nets de trésorerie générés par l'activité (a) en 2023 sont de 123 231 947 milliers de FCFA."}

In [ ]:
question = "resumer de ces fichiers en une phrase "
qa_chain({"query": question})

{'query': 'resumer de ces fichiers en une phrase ',
 'result': "Les premiers résultats sont encourageants avec une amélioration de la tendance de l'EBITDAaL, ce qui renforce la confiance dans la capacité à atteindre les objectifs du plan Lead the Future."}